In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import streamlit as st
import openai
from openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import create_qa_with_sources_chain
from langchain.chains import (
            StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
        )
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI


In [2]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
OpenAI_key = os.environ.get("opena_api_key")

In [4]:
from openai import OpenAI
client = OpenAI(api_key=OpenAI_key)

audio_file = open("test.m4a", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)


In [5]:
transcription

"Hello ladies and gentlemen, how are you doing? My name is Timothy. I want to try using WispR, you know, I tried it last time when I went to shoot FFMVorever, but now I think it will go ahead, right? So, I'm recording this, I'm going to save it to reformat, or if possible in MP3 format. Then I'm going to test it to see the transcription, which it actually worked really fine, right? Alright, so the end goal of this project is to create a 4-stroke orchestral system, right? So I need to be very fast, so possible ways of doing that is what I need right now, okay? Alright, so thank you for listening, and stay blessed. Bye bye!\n"

In [4]:
df = pd.read_excel("240305 Muchacha menu v1.xlsx")
df.head(10)

,id_client,client,category,section,id_item,item,description,price_dinein,price_delivery,price_pickup,available_dinein,available_delivery,available_pickup,dietary,contains,calories,prep_time,alcohol
0,1,Muchacha,Food,SHARE AROUND,1,Freshly Cooked Tortilla Corn Chips,NaN,5.5,6.5,6.5,True,True,True,"GF, V",NaN,300,20,False
1,1,Muchacha,Food,SHARE AROUND,2,Guacamole (small),"Avocado, Coriander & Lime Juice",6.0,7.0,7.0,True,True,True,"GF, V",NaN,150,20,False
2,1,Muchacha,Food,SHARE AROUND,3,Guacamole (large),"Avocado, Coriander & Lime Juice",10.0,11.0,11.0,True,True,True,"GF, V",NaN,300,20,False
3,1,Muchacha,Food,SHARE AROUND,4,Pico De Gallo Salsa (small),"w/ Tomato, Onion, Jalapeños, Coriander & Lime ...",5.0,6.0,6.0,True,True,True,"GF, V",NaN,25,20,False
4,1,Muchacha,Food,SHARE AROUND,5,Pico De Gallo Salsa (large),"w/ Tomato, Onion, Jalapeños, Coriander & Lime ...",8.0,9.0,9.0,True,True,True,"GF, V",NaN,50,20,False
5,1,Muchacha,Food,SHARE AROUND,6,Jalapeño Frickles,Crispy Fried Pickled Jalapeños w/ Citru s Yoghurt,7.0,8.0,8.0,True,True,True,"GF, V",NaN,450,20,False
6,1,Muchacha,Food,SHARE AROUND,7,Pork Taquitos,Cheese & Pork Corn Tortilla ‘Cigars’ Fried w/ ...,16.0,17.0,17.0,True,True,True,GF,NaN,500,20,False
7,1,Muchacha,Food,SHARE AROUND,8,Veggie Taquitos,"Cheese, Corn, Black Beans, Roasted Capsicum Co...",16.0,17.0,17.0,True,True,True,"GF, V",NaN,400,20,False
8,1,Muchacha,Food,SHARE AROUND,9,Sweet Potato Fries,w/ Chipotle BBQ Sauce & Paprika & Ha banero Salt,10.0,11.0,11.0,True,True,True,"GF, V",NaN,400,20,False
9,1,Muchacha,Food,SHARE AROUND,10,Spicy Buffalo Chicken Wings,NaN,15.0,16.0,16.0,True,True,True,GF,Hot,600,20,False


In [5]:
df.shape

(65, 18)

In [6]:
df.isna().sum()

id_client              0
client                 0
category               0
section                0
id_item                0
item                   0
description           32
price_dinein           0
price_delivery         0
price_pickup           0
available_dinein       0
available_delivery     0
available_pickup       0
dietary               27
contains              61
calories               0
prep_time              0
alcohol                0
dtype: int64

In [7]:
df.columns

Index(['id_client', 'client', 'category', 'section', 'id_item', 'item',
       'description', 'price_dinein', 'price_delivery', 'price_pickup',
       'available_dinein', 'available_delivery', 'available_pickup', 'dietary',
       'contains', 'calories', 'prep_time', 'alcohol'],
      dtype='object')

In [8]:
df.shape

(65, 18)

In [9]:
cols_to_drop = ['id_client', 'client', 'contains', 'dietary']
df_1 = df.drop(cols_to_drop, axis=1)

In [10]:
# filling missing values with not available
df_1 = df_1.fillna("Not Available at the moment")

In [11]:
df_1.head(10)

,category,section,id_item,item,description,price_dinein,price_delivery,price_pickup,available_dinein,available_delivery,available_pickup,calories,prep_time,alcohol
0,Food,SHARE AROUND,1,Freshly Cooked Tortilla Corn Chips,Not Available at the moment,5.5,6.5,6.5,True,True,True,300,20,False
1,Food,SHARE AROUND,2,Guacamole (small),"Avocado, Coriander & Lime Juice",6.0,7.0,7.0,True,True,True,150,20,False
2,Food,SHARE AROUND,3,Guacamole (large),"Avocado, Coriander & Lime Juice",10.0,11.0,11.0,True,True,True,300,20,False
3,Food,SHARE AROUND,4,Pico De Gallo Salsa (small),"w/ Tomato, Onion, Jalapeños, Coriander & Lime ...",5.0,6.0,6.0,True,True,True,25,20,False
4,Food,SHARE AROUND,5,Pico De Gallo Salsa (large),"w/ Tomato, Onion, Jalapeños, Coriander & Lime ...",8.0,9.0,9.0,True,True,True,50,20,False
5,Food,SHARE AROUND,6,Jalapeño Frickles,Crispy Fried Pickled Jalapeños w/ Citru s Yoghurt,7.0,8.0,8.0,True,True,True,450,20,False
6,Food,SHARE AROUND,7,Pork Taquitos,Cheese & Pork Corn Tortilla ‘Cigars’ Fried w/ ...,16.0,17.0,17.0,True,True,True,500,20,False
7,Food,SHARE AROUND,8,Veggie Taquitos,"Cheese, Corn, Black Beans, Roasted Capsicum Co...",16.0,17.0,17.0,True,True,True,400,20,False
8,Food,SHARE AROUND,9,Sweet Potato Fries,w/ Chipotle BBQ Sauce & Paprika & Ha banero Salt,10.0,11.0,11.0,True,True,True,400,20,False
9,Food,SHARE AROUND,10,Spicy Buffalo Chicken Wings,Not Available at the moment,15.0,16.0,16.0,True,True,True,600,20,False


In [31]:
df_1.category.value_counts()[1]

C:\Users\timmy\AppData\Local\Temp\ipykernel_20716\30598450.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_1.category.value_counts()[1]


13

Item_id is {id_item}. Item name (or food name) is {item}. All informations regarding {item} is as follow.
The description of the item is {description}, it is available for for dining if you wish to dine in. Available for delivery ? : {available_delivery}. Available for pickup? {available_pickup}. It has {calories} much of calories in it. It will take {prep_time} to prepare it. Does it have alcohol in it? {alcohol}. 

In [26]:
# creating a function that reads the excel file and then creates a document file from it
def read_excel_file(file):
    df = pd.read_excel(file)
    cols_to_drop = ['id_client', 'client', 'contains', 'dietary']
    df_1 = df.drop(cols_to_drop, axis=1)
    # filling missing values with "Not Available at the moment"
    df_1 = df_1.fillna("Not Available at the moment")
    # writing the data to a text file
    with open('menu_items.txt', 'w') as file:
        file.write("Menu Items\n\n")
        for _, row in df_1.iterrows():
            item_id = row['id_item']
            item_name = row['item']
            category = row['category']
            description = row['description']
            available_delivery = row['available_delivery']
            available_pickup = row['available_pickup']
            calories = row['calories']
            prep_time = row['prep_time']
            alcohol = row['alcohol']
            file.write(f"Item_id is {item_id}. It is in the {category} category. Item name (or food name) is {item_name}. All information regarding {item_name} is as follows.\n")
            file.write(f"The description of the item is {description}, it is available for dining if you wish to dine in.\n")
            file.write(f"Available for delivery? : {available_delivery}.\n")
            file.write(f"Available for pickup? {available_pickup}.\n")
            file.write(f"It has {calories} calories in it.\n")
            file.write(f"It will take {prep_time} to prepare it.\n")
            file.write(f"Does it have alcohol in it? {alcohol}.\n\n")
    
    return "Text file created successfully"


# using the function
read_excel_file("240305 Muchacha menu v1.xlsx")

'Text file created successfully'

In [34]:
# creating another one for items_id and items
def items(file):
    df = pd.read_excel(file)
    cols_to_drop = ['id_client', 'client', 'contains', 'dietary']
    df_1 = df.drop(cols_to_drop, axis=1)
    # filling missing values with "Not Available at the moment"
    df_1 = df_1.fillna("Not Available at the moment")
    # writing the data to a text file
    with open('id_items.txt', 'w') as file:
        file.write("Id Items\n\n")
        for _, row in df_1.iterrows():
            item_id = row['id_item']
            item_name = row['item']
            file.write(f"{item_id} : {item_name} \n")

items("240305 Muchacha menu v1.xlsx")         

In [27]:

# Step 1
raw_documents = TextLoader("menu_items.txt").load()

In [28]:
# Step 2
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, length_function=len
)
documents = text_splitter.split_documents(raw_documents)

In [30]:
documents[0]

Document(page_content='Menu Items\n\nItem_id is 1. It is in the Food category. Item name (or food name) is Freshly Cooked Tortilla Corn Chips. All information regarding Freshly Cooked Tortilla Corn Chips is as follows.\nThe description of the item is Not Available at the moment, it is available for dining if you wish to dine in.\nAvailable for delivery? : True.\nAvailable for pickup? True.\nIt has 300 calories in it.\nIt will take 20 to prepare it.\nDoes it have alcohol in it? False.', metadata={'source': 'menu_items.txt'})

In [24]:
# Step 3
embeddings_model = OpenAIEmbeddings(api_key=OpenAI_key)
db = FAISS.from_documents(documents, embeddings_model)

# Step 4
retriever = db.as_retriever()

c:\Users\timmy\Downloads\Alex proj\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [41]:
from langchain_openai import ChatOpenAI
# Step 5
llm_src = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106", api_key=OpenAI_key)
qa_chain = create_qa_with_sources_chain(llm_src)
retrieval_qa = ConversationalRetrievalChain.from_llm(
    llm_src,
    retriever,
    return_source_documents=True,
)


In [42]:
# Output
output = retrieval_qa({
    "question": "What are the available Tacos",
    "chat_history": []
})

In [43]:
print(f"Question: {output['question']}")
print(f"Answer: {output['answer']}")
print(f"Source: {output['source_documents'][0].metadata['source']}")

Question: What are the available Tacos
Answer: The available tacos are the Prawn Tacos and the description is "Grilled Large Flour Tortilla & Cheese Served w/ Side of Salsa & Citrus."
Source: menu_items.txt
